In [ ]:
using PyPlot
using Random
using LinearAlgebra

Our goal is, given a function $f:X\times Y \to \mathbf{R}$, to find the mixed Nash equilibria, that is solutions to 
$$\min_{\mu \in \mathcal{P}(X)} \max_{\nu \in \mathcal{P}(Y)} \int f(x,y)d\mu(X)d\nu(x)$$
Let us take $X=Y=\mathbb{T}$ the torus and a random smooth function $f$ (which, for convenience, we parameterize by its Fourier coefficients).

## Define the algorithms

Definition of the particle gradient methods :
- sym-GDA
- alt-GDA
- extra-gradient

In [ ]:
"""
Find mixed Nash equilibria via particle (simultaneous) gradient descent ascent 
    - p0,q0 : initial weights
    - x0,y0 : initial positions
    - c : weights of the Gaussian process
    - η : step-size
"""
function particle_symGDA(p0, q0, x0, y0, c, η, niter;  α = 1)
    m, n = length(p0), length(q0)
    nf = div(size(c,1)-1,2) #nb of frequencies
    f(x1,x2)   = real(sum(c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂1f(x1,x2) = real(sum(2π*im*k1*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂2f(x1,x2) = real(sum(2π*im*k2*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ps, qs = zeros(m,niter), zeros(n,niter)
    xs, ys = zeros(m,niter), zeros(n,niter)
    p,q,x,y = p0,q0,x0,y0
    for t = 1:niter
        ps[:,t], qs[:,t], xs[:,t], ys[:,t] = p, q, x, y

        A  = f.(x,y')
        Ax = ∂1f.(x,y')
        Ay = ∂2f.(x,y')
        pp = p.*exp.(-η*A*q)
        qq = q.*exp.(η*A'*p)
        x = x - α*η*Ax*q
        y = y + α*η*Ay'*p
        p = pp/sum(pp)
        q = qq/sum(qq) 
    end
    return ps, qs, xs, ys
end

In [ ]:
"""
Find mixed Nash equilibria via particle alternating gradient descent ascent 
    - p0,q0 : initial weights
    - x0,y0 : initial positions
    - c : weights of the Gaussian process
    - η : step-size
"""
function particle_altGDA(p0, q0, x0, y0, c, η, niter;  α = 1)
    m, n = length(p0), length(q0)
    nf = div(size(c,1)-1,2) #nb of frequencies
    f(x1,x2)   = real(sum(c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂1f(x1,x2) = real(sum(2π*im*k1*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂2f(x1,x2) = real(sum(2π*im*k2*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ps, qs = zeros(m,niter), zeros(n,niter)
    xs, ys = zeros(m,niter), zeros(n,niter)
    p,q,x,y = p0,q0,x0,y0
    for t = 1:niter
        ps[:,t], qs[:,t], xs[:,t], ys[:,t] = p, q, x, y

        # update one side
        A = f.(x,y')
        p = p.*exp.(-η*A*q)
        p = p/sum(p)
        Ax= ∂1f.(x,y')
        x = x - α*η*Ax*q
        
        # update other side
        B = f.(x,y')
        q = q.*exp.(η*B'*p)
        q = q/sum(q)
        By= ∂2f.(x,y')
        y = y + α*η*By'*p
    end
    return ps, qs, xs, ys
end

In [ ]:
"""
Find mixed Nash equilibria via particle extragradient
    - p0,q0 : initial weights
    - x0,y0 : initial positions
    - c : weights of the Gaussian process
    - η : step-size
"""
function particle_EG(p0, q0, x0, y0, c, η, niter;  α = 1)
    m, n = length(p0), length(q0)
    nf = div(size(c,1)-1,2) #nb of frequencies
    f(x1,x2)   = real(sum(c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂1f(x1,x2) = real(sum(2π*im*k1*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂2f(x1,x2) = real(sum(2π*im*k2*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ps, qs = zeros(m,niter), zeros(n,niter)
    xs, ys = zeros(m,niter), zeros(n,niter)
    p,q,x,y = p0,q0,x0,y0
    for t = 1:niter
        ps[:,t], qs[:,t], xs[:,t], ys[:,t] = p, q, x, y

        # gradient
        A  = f.(x,y')
        Ax = ∂1f.(x,y')
        Ay = ∂2f.(x,y')
        pp = p.*exp.(-η*A*q)
        pp = pp/sum(pp)
        qq = q.*exp.(η*A'*p)
        qq = qq/sum(qq)
        xx = x - α*η*Ax*q
        yy = y + α*η*Ay'*p
        
        # extragradient
        B  = f.(xx,yy')
        Bx = ∂1f.(xx,yy')
        By = ∂2f.(xx,yy')
        p = p.*exp.(-η*B*qq)
        p = p/sum(p)
        q = q.*exp.(η*B'*pp)
        q = q/sum(q)
        x = x - α*η*Bx*qq
        y = y + α*η*By'*pp
    end
    return ps, qs, xs, ys
end

## Experiment for the over-parameterized case

Define a random payoff function

In [ ]:
Random.seed!(3)#3#7
nf = 4 # there are (nf+1)^d (real-valued) basis functions
c = randn(2nf+1,2nf+1) + im*randn(2nf+1,2nf+1); # random fourier coeffs

# plot
figure(figsize=[3,3])
xs = 0:0.01:1
f(x1,x2) = real(sum(c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
pcolor(xs,xs',f.(xs,xs'), shading="auto");axis("off");#title(L"f");
#savefig("potential.png",dpi=100,bbox_inches="tight")

In [ ]:
η = 0.1
niter = 4000
m, n = 30, 20 
p0, q0 = ones(m)/m, ones(n)/n
x0 = range(0,1,length=m)
y0 = range(0,1,length=n);
@time ps, qs, xs, ys = particle_EG(p0, q0, x0, y0, c, η, niter;  α = 0.05);

Plot the trajectories (remember that we are on the torus, here the torus is not "folded")

In [ ]:
figure(figsize=[10,4])
subplot(121)
plot(xs',ps',"k");
plot(xs[:,end],ps[:,end],"or");
title("Player A"); xlabel("position"); ylabel("weight");

subplot(122)
plot(ys',qs',"k");
plot(ys[:,end],qs[:,end],"or");
title("Player B"); xlabel("position"); ylabel("weight");

### Code to make a nice movie

In [ ]:
#(m,n) = (mm,nn) #TEMP
ts = Int.(floor.((range(1,niter,length=500))))
i=10
tl = 15
xs = mod.(xs,1)
ys = mod.(ys,1)
for i=1:length(ts)
    close()
    figure(figsize=[8,3])
    subplot(121)
    plot(xs[:,ts[i]],ps[:,ts[i]],"oC0")
    for j=1:m
        if abs(xs[j,ts[i]]-xs[j,maximum([ts[i]-tl,1])])<0.5
            plot(xs[j,maximum([ts[i]-tl,1]):ts[i]],ps[j,maximum([ts[i]-tl,1]):ts[i]],"k")
        end
    end
        #plot(xs[j,maximum([ts[i]-tl,1]):ts[i]],ps[j,maximum([ts[i]-tl,1]):ts[i]],"k")
    #plot(xsol,psol,"xr") #TEMP
    xlabel("position")
    ylabel("weight")
    yticks([])
    xticks([-0,1])
    axis([0.0,1.0,0,maximum(ps)+0.1])
    title("Player A")

    subplot(122)
    plot(ys[:,ts[i]],qs[:,ts[i]],"oC1")
    for j=1:n
        if abs(ys[j,ts[i]]-ys[j,maximum([ts[i]-tl,1])])<0.5
            plot(ys[j,maximum([ts[i]-tl,1]):ts[i]],qs[j,maximum([ts[i]-tl,1]):ts[i]],"k")
        end
    end
    #plot(ysol,qsol,"xr") #TEMP
    xlabel("position")
    ylabel("weight")
    yticks([])
    xticks([-0,1])
    axis([0.0,1.0,0,maximum(qs)+0.1])
    title("Player B")
    
    #savefig("output/CP-EG-yes/CP-EG-yes-$(i).png",dpi=200,bbox_inches="tight")
    #savefig("output/EP-AG-yes/EP-AG-yes-$(i).png",dpi=200,bbox_inches="tight")
end

## Prepare experiments in the exact parameterized case

In [ ]:
"Extract the positions of the non-zero particles (and clustering)"
function extract(x, p; tol=1e-4)
    xx, pp  = [mod1(x[1],1)], [p[1]]
    for k = 2:length(x) 
        (v,ind) = findmin(abs.(xx .- mod1.(x[k],1)))
        if v > tol
            append!(xx,mod1.(x[k],1))
            append!(pp,p[k])
        else
            pp[ind] += p[k]
        end
    end
    return xx[pp .> tol], pp[pp .> tol]
end

In [ ]:
# create clusters of positions at 10^(-6) precision
xx,pp = extract(xs[:,end],ps[:,end])
yy,qq = extract(ys[:,end],qs[:,end])
mm, nn = length(xx), length(yy)
println("Number of particles of the mixed Nash equilibrium (Player A, Player B) = (",mm,", ",nn,")")

Compute the true saddle to a high precision

In [ ]:
η = 0.1
niter = 30000
p0, q0 = pp, qq 
ps, qs, xs, ys = particle_EG(p0, q0, xx, yy, c, η, niter;  α = 0.005);
psol, qsol, xsol, ysol = ps[:,end], qs[:,end], xs[:,end], ys[:,end]

## Compute the Jacobian at the saddle point

We start with a function needed in the definition of the Jacobian (see paper).

In [ ]:
"""
Given an input vector v of size d, creates a orthonormal matrix which first column is v/||v||_2
"""
function ONbasiscomplete(v)
    d=length(v)
    Q = Matrix(1.0*I, d, d)
    for k=2:length(v)
        Pv = dot(v,Q[:,1])*Q[:,1] + dot(v,Q[:,k])*Q[:,k] # Project v on the plane (e1,ek)
        Pv = Pv ./sqrt(sum(Pv.^2)) # normalize to length 1
        cosθ = dot(Pv,Q[:,1])
        sinθ = dot(Pv,Q[:,k])
        Qk = Matrix(1.0*I, d, d)
        Qk[1,1] = cosθ; Qk[k,1] = sinθ
        Qk[1,k] = -sinθ; Qk[k,k] = cosθ
        Q = Q*Qk
    end
    return Q
end

d=4
v = randn(d)
v = v ./ sqrt(sum(v.^2))
Q = ONbasiscomplete(v)
display(v), display(Q),display(Q'*Q),display(Q*Q');

Compute the Jacobian of the continuous time dynamics at the saddle point

In [ ]:
"""
Compute the Jacobian of gradient flow at the saddle and return 
(i) the minimum real part of eigenvalues (removing the two zeros)
(ii) the value predicted by the expansion for α small
Function is only correct when the input corresponds to a saddle (otherwise last line needs change)
"""
function compute_rate_GF(pp,qq,xx,yy,c, α)
    f(x1,x2)    = real(sum(c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂1f(x1,x2)  = real(sum(2π*im*k1*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂2f(x1,x2)  = real(sum(2π*im*k2*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂12f(x1,x2) = real(sum(-4π^2*k2*k1*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂11f(x1,x2) = real(sum(-4π^2*k1*k1*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))
    ∂22f(x1,x2) = real(sum(-4π^2*k2*k2*c[k1+nf+1,k2+nf+1]*exp(2π*im*(k1*x1+k2*x2))/(1+k1^2+k2^2) for k1 in -nf:nf, k2 in -nf:nf))

    P  = [f(x,y) for x in xx, y in yy]
    P1 = [∂1f(x,y) for x in xx, y in yy]
    P2 = [∂2f(x,y) for x in xx, y in yy]
    P12= [∂12f(x,y) for x in xx, y in yy]
    P11= [∂11f(x,y) for x in xx, y in yy]
    P22= [∂22f(x,y) for x in xx, y in yy]

    mm  = length(pp)
    nn  = length(qq)
    Dp  = diagm(sqrt.(pp))
    Dq  = diagm(sqrt.(qq))
    Kp  = ONbasiscomplete(sqrt.(pp))
    Kq  = ONbasiscomplete(sqrt.(qq))
    PP1 = Kp'*Dp*P*Dq*Kq
    PP2 = sqrt(α)*Kp'*Dp*P2*Dq
    PP3 = sqrt(α)*Dp*P1*Dq*Kq
    PP4 = α*Dp*P12*Dq
    PP5 = α*diagm(P22'*pp)
    PP6 = α*diagm(P11*qq)
    DD = diagm(cat(0,ones(mm-1), ones(mm), 0, ones(nn-1),ones(nn),dims=1))
    
    M = DD*cat(cat(zeros(mm,mm),zeros(mm,mm),PP1         ,PP2         , dims=2),
            cat(zeros(mm,mm),PP6         ,PP3         ,PP4         , dims=2),
            cat(-PP1'       ,-PP3'       ,zeros(nn,nn),zeros(nn,nn), dims=2),
            cat(-PP2'       ,-PP4'       ,zeros(nn,nn),-PP5         , dims=2),
        dims=1)*DD
    Ptrue = M[1:2mm,2mm+1:end]
    Qtrue = M[1:2mm,1:2mm]
    Rtrue = M[2mm+1:end,2mm+1:end]

    U = svd(Ptrue).U
    V = svd(Ptrue).V
    #[U[:,j]'*Qtrue*U[:,j] + V[:,j]'*Rtrue*V[:,j] for j in 1:2mm]
    #@show size(U), size(Qtrue), size(V), size(Rtrue)

    mod_sa = minimum(setdiff(real.(eigen(M).values),0))  #remove the two first 0 eigenvalues
    mod_appr = minimum(setdiff([U[:,j]'*Qtrue*U[:,j] + V[:,j]'*Rtrue*V[:,j] for j in 1:size(U,2)],0))
    return mod_sa, mod_appr/2#, tr(M)/(mm+nn)
end

## Comparison of convergence rates for particle dynamics

In [ ]:
niter = 10000
Random.seed!(2)
p0, q0 = psol .+ 0.01*rand(size(psol)), qsol .+ 0.01*rand(size(qsol)) #only look at local convergence
#################### SWITCH THESE LINES OF CODE TO DO ONE OR THE OTHER GRAPH
# αs = 10.0 .^ range(-20,-2,length=2) # fixed alphas
# ηs = 10.0 .^ range(-2.5,0,length=40) # fixed alphas
αs = 10.0 .^ range(-3,-1,length=40) # fixed etas
ηs = 10.0 .^ range(-2,-2,length=1) # fixed etas
####################
rate_altGDA = zeros(length(αs ),length(ηs))
rate_symGDA = zeros(length(αs ),length(ηs))
rate_EG = zeros(length(αs ),length(ηs))
(i,j)=(1,1)
@time for i=1:length(αs)
        for j=1:length(ηs)
        α = αs[i]
        η = ηs[j]
        # sym-GDA
        ps, qs, xs, ys = particle_symGDA(p0, q0, xx, yy, c, η, niter;  α = α);
        err = sqrt.(sum((ps[:,:] .- psol[:]).^2, dims=1) .+ sum((qs[:,:] .- qsol[:]).^2, dims=1).+ sum((xs[:,:] .- xsol[:]).^2, dims=1).+ sum((ys[:,:] .- ysol[:]).^2, dims=1))';
        err = err[err.>1e-10]
        err=err[2:end]
        niter_eff = length(err)
        a = [ones(niter_eff) 1:niter_eff] \ log.(err)
        rate_symGDA[i,j]=a[2]#/η
        
        # alt-GDA
        ps, qs, xs, ys = particle_altGDA(p0, q0, xx, yy, c, η, niter;  α = α);
        err = sqrt.(sum((ps[:,:] .- psol[:]).^2, dims=1) .+ sum((qs[:,:] .- qsol[:]).^2, dims=1).+ sum((xs[:,:] .- xsol[:]).^2, dims=1).+ sum((ys[:,:] .- ysol[:]).^2, dims=1))';
        err = err[err.>1e-10]
        err=err[2:end]
        niter_eff = length(err)
        a = [ones(niter_eff) 1:niter_eff] \ log.(err)
        rate_altGDA[i,j]=a[2]#/η
        
        # EG
        ps, qs, xs, ys = particle_EG(p0, q0, xx, yy, c, η, niter;  α = α);
        err = sqrt.(sum((ps[:,:] .- psol[:]).^2, dims=1) .+ sum((qs[:,:] .- qsol[:]).^2, dims=1).+ sum((xs[:,:] .- xsol[:]).^2, dims=1).+ sum((ys[:,:] .- ysol[:]).^2, dims=1))';
        err = err[err.>1e-10]
        err=err[2:end]
        niter_eff = length(err)
        a = [ones(niter_eff) 1:niter_eff] \ log.(err)
        rate_EG[i,j]=a[2]#/η
        end
end


In [ ]:
# gradient flow (theoretical)
rate_GF = zeros(length(αs))
rate_GF_asymp = zeros(length(αs))
for i = 1:length(αs)
    rate_GF[i], rate_GF_asymp[i] = compute_rate_GF(pp,qq,xx,yy,c, αs[i])
end

In [ ]:
figure(figsize=[5,3])
loglog(αs,-rate_symGDA[:,end],lw=3,label="Sim-GDA","o", alpha=0.75)
loglog(αs,-rate_EG[:,end],lw=3,label="EG","o", alpha=0.75)
loglog(αs,-rate_altGDA[:,end],lw=3,label="Alt-GDA","oC3", alpha=0.75)
loglog(αs, ηs[end]*rate_GF.* ones(length(αs)),lw=1,label="GF (theory)","k")
#loglog(αs, 500*αs.^2,lw=1,label="GF (theory)","k")

legend()
xlabel(L"\gamma")
ylabel("Convergence rate")
#title(L"Fixed step-size ($\eta=10^{-2}$)")
grid("on")
savefig("rate_vs_alpha.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[5,3])
loglog(ηs,-rate_symGDA[end,:],lw=3,label="Sim-GDA","o", alpha=0.75)
loglog(ηs,-rate_EG[end,:],lw=3,label="EG","o", alpha=0.75)
loglog(ηs,-rate_altGDA[end,:],lw=3,label="Alt-GDA","oC3", alpha=0.75)
loglog(ηs,-rate_EG[1,:],lw=3,label=L"EG ($\gamma=0$)","o", alpha=0.75)
loglog(ηs, rate_GF[end]*ηs,lw=1,label="GF (theory)","k")
#loglog(αs, 500*αs.^2,lw=1,label="GF (theory)","k")

legend()
xlabel(L"\eta")
ylabel("Convergence rate")
#title(L"Fixed $\gamma$ ($\gamma=10^{-2}$)")
grid("on")
savefig("rate_vs_eta.png",bbox_inches="tight",dpi=200)